In [ ]:
import csv
import logging
from db import DB

# Initialize your database connection (assuming DB is your ORM class)
db = DB()

# Path to your CSV file
csv_file = 'data/with_ofss_ids.csv'

def load_csv(file_path):
    """
    Loads the data from a CSV file.

    Parameters:
    - file_path (str): Path to the CSV file.

    Returns:
    - list: Parsed data from the CSV file as a list of dictionaries.
    """
    data = []
    with open(file_path, 'r') as f:
        reader = csv.DictReader(f)
        for row in reader:
            data.append(row)
    return data

def upsert_financial_data(db, csv_data):
    """
    Upserts the financial data into the database.

    Parameters:
    - db: The DB instance from your ORM.
    - csv_data (list): The parsed CSV data.
    """
    try:
        for row in csv_data:
            tag = row['tag']

            if tag.endswith("Text"):
                logging.debug(f"Skipping tag: {tag}")
                continue

            statement_type = row['statement_type'] if row['statement_type'] else None
            balance = row['balance'] if row['balance'] else None
            period_type = row['period_type'] if row['period_type'] else None

            ofss_id = row['ofss_id'] if row['ofss_id'] else None

            if balance is not None:
                # Upsert balance type
                balance_type_id = db.upsert_entity('us_gaap_balance_type', {'balance': balance}, ['balance'])
            else:
                balance_type_id = None
            
            if period_type is not None:
                # Upsert period type
                period_type_id = db.upsert_entity('us_gaap_period_type', {'period_type': period_type}, ['period_type'])
            else:
                period_type_id = None
            
            if statement_type is not None:
                # Upsert statement type
                statement_type_id = db.upsert_entity('us_gaap_statement_type', {'statement_type': statement_type}, ['statement_type'])
            else:
                statement_type_id = None
            
            # Upsert us_gaap_concept (the tag itself)
            tag_data = {
                'name': tag,
                'balance_type_id': balance_type_id,
                'period_type_id': period_type_id,
            }
            tag_id = db.upsert_entity('us_gaap_concept', tag_data, ['name'])

            if ofss_id is not None:
                # Aassociate tag with the ofss category
                db.upsert_entity('us_gaap_concept_ofss_category', {
                    'us_gaap_concept_id': tag_id,
                    'ofss_category_id': ofss_id,
                    'is_manually_mapped': True # Hardcoded
                }, ['us_gaap_concept_id', 'ofss_category_id'])

                # Note: This is ideal, but must know all category ids first
                # Upsert associations for `us_gaap_concept_ofss_category`
                # db.upsert_and_cleanup(
                #     table_name="us_gaap_concept_ofss_category",
                #     entity_id_name="us_gaap_concept_id",
                #     parent_id_name="ofss_category_id",
                #     parent_id=tag_id,
                #     associate_entities=[{'ofss_category_id': ofss_category_id} for ofss_category_id in [ofss_id]],
                #     upsert_datetime_field=None,
                #     unique_fields=["us_gaap_concept_id", "ofss_category_id"]
                # )

                if statement_type_id is not None:
                    # Associate tag with statement type
                    db.upsert_entity('us_gaap_concept_statement_type', {
                        'us_gaap_concept_id': tag_id,
                        'us_gaap_statement_type_id': statement_type_id,
                        'is_manually_mapped': True # Hardcoded
                    }, ['us_gaap_concept_id', 'us_gaap_statement_type_id'])

                    # Note: This is ideal, but must know all statement ids first
                    # db.upsert_and_cleanup(
                    #     table_name="us_gaap_concept_statement_type",
                    #     entity_id_name="us_gaap_concept_id",
                    #     parent_id_name="us_gaap_statement_type_id",
                    #     parent_id=tag_id,
                    #     associate_entities=[{'us_gaap_statement_type_id': statement_type_id} for statement_type_id in [statement_type_id]],
                    #     upsert_datetime_field=None,
                    #     unique_fields=["us_gaap_concept_id", "us_gaap_statement_type_id"]
                    # )

            logging.debug(f"Upserted data for tag: {tag} with ofss_id: {ofss_id}")

        logging.info('Financial statement data has been successfully upserted.')
    except Exception as e:
        logging.error(f"Error upserting financial statement data: {e}")
        raise

if __name__ == "__main__":
    # Load the CSV data
    csv_data = load_csv(csv_file)

    # Upsert the data into the database
    upsert_financial_data(db, csv_data)
